<a href="https://colab.research.google.com/github/changyuhsin1999/duke_chatbot/blob/main/re_embed_data_to_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone 'https://github.com/changyuhsin1999/duke_chatbot.git'
!pip install -U bitsandbytes
!pip install transformers==4.36.2
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece
!pip install openpyxl
!pip install xlrd
!pip install openai
!pip install huggingface_hub
!pip install pinecone-client
!pip install langchain
!pip install sentence_transformers
!pip install python-dotenv

fatal: destination path 'duke_chatbot' already exists and is not an empty directory.
  Using cached sentence_transformers-2.6.1-py3-none-any.whl (163 kB)


In [11]:
import os
import torch
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer, AutoModel
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec


In [12]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [6]:
import json

# Load the JSON data
input_file_path = 'duke_chatbot/data/extracted_data_2024-04-01_07-59-36.json'
cleaned_file_path = 'duke_chatbot/data/cleaned_data.json'

text_to_replace = ["Duke AI Master of Engineering Jump to navigation Duke Engineering Pratt School of Engineering Institute for Enterprise Engineering Industry Relations Leadership News Contact Why Duke? The Duke Difference Career Services Graduate Outcomes What Tech Leaders Are Saying Degree Certificate Courses Faculty Apply",
                   "Join our mailing list! Sign up to receive more information about Duke Engineering graduate programs and recruitment events » Admissions Have a question about the program or admissions? Contact the Admissions & Recruiting team at pratt_masters@duke.edu . © Copyright 2011-2024 Duke University Main Menu Why Duke? The Duke Difference Career Services Graduate Outcomes What Tech Leaders Are Saying Degree Certificate Courses Faculty Apply Quick Links Industry Relations Leadership News Contact"]

def clean_text(original_text, texts_to_remove):
    for text in texts_to_remove:
        original_text = original_text.replace(text, "")  # Replace with an empty string to remove
    return original_text

# Read the JSON data
with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Iterate through each URL entry and clean the text
cleaned_data = {}
for url, text in data.items():
    cleaned_text = clean_text(text, text_to_replace)
    cleaned_data[url] = cleaned_text

# Save the cleaned data to a new JSON file
with open(cleaned_file_path, 'w', encoding='utf-8') as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print(f"Cleaned data saved to {cleaned_file_path}")


Cleaned data saved to duke_chatbot/data/cleaned_data.json


In [13]:
import json

class CharacterTextSplitter:
    def __init__(self, chunk_size, chunk_overlap):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def split_text(self, text):
        # Split the text into chunks with specified size and overlap
        chunks = [text[i:i+self.chunk_size] for i in range(0, len(text), self.chunk_size - self.chunk_overlap)]
        return chunks

class DocumentProcessor:
    def __init__(self, text_splitter):
        self.text_splitter = text_splitter
        self.total_chunks = 0

    def process_faq_json(self, file_path):
        # Read the JSON file
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)

        chunks = []
        for entry in data.get('FAQs', []):
            text = f"Question: {entry['question']}\nAnswer: {entry['answer']}"
            text_chunks = self.text_splitter.split_text(text)
            chunks.extend(text_chunks)
            self.total_chunks += len(text_chunks)

        return chunks

    def process_extracted_json(self, file_path):
        # Read the JSON file
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)

        chunks = []
        for text in data.items():
            text_chunks = self.text_splitter.split_text(text)
            chunks.extend(text_chunks)
            self.total_chunks += len(text_chunks)

        return chunks

    def process_finetune_qa(self, file_path):
        # Read the JSON file
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)

        chunks = []
        for item in data:
            combined_text = f"Input: {item['input']}\nOutput: {item['output']}"
            chunks.append(combined_text)
            self.total_chunks += 1

        return chunks

# Example usage
text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=25)
processor = DocumentProcessor(text_splitter=text_splitter)
chunks_extracted = processor.process_extracted_json('duke_chatbot/data/cleaned_data.json')
chunks_faq = processor.process_faq_json('duke_chatbot/data/extracted_data_from_faq.json')
chunks_finetune_qa = processor.process_finetune_qa('duke_chatbot/data/fine_tune_data.json')
all_chunks = chunks_extracted + chunks_faq + chunks_finetune_qa

print(f"Total chunks processed: {processor.total_chunks}")

Total chunks processed: 818


In [14]:
print(all_chunks)

[('https://ai.meng.duke.edu/faculty/wann-jiun-ma', 'Wann-Jiun Ma |  Wann-Jiun Ma Adjunct Assistant Professor in the Engineering Graduate and Professional Programs Wann-Jiun received his PhD in Electrical Engineering from University of Notre Dame in 2015, and then worked as a postdoctoral associate at Duke University. He has been working for both big and startup companies as a data scientist. Currently, he is the Director of Data Science/AI at Neiman Marcus Group. In addition to being a data science practitioner in the industry, he also collaborates with multiple data science MS programs and guides students as they work on their deep reinforcement learning capstone projects. His current research interests are deep learning/deep reinforcement learning/large language models and their real-life applications, especially product recommendations and personalization. Appointments and Affiliations Adjunct Assistant Professor in the Engineering Graduate and Professional Programs Contact Informat

In [15]:
from google.colab import userdata
PINECONE_API_KEY = userdata.get('Pinecone')
OPENAI_API_KEY = userdata.get('OpenAI')

In [16]:
class Embedder:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed_text_chunks(self, chunks):
        embeddings = []
        for chunk in chunks:
            # Tokenize the chunk and add special tokens at the beginning and end
            inputs = self.tokenizer(chunk, return_tensors="pt", padding=True, truncation=True, max_length=512)
            # Pass the tokenized inputs to the model
            with torch.no_grad():
                outputs = self.model(**inputs)
            # Get the embeddings from the last hidden state
            chunk_embeddings = outputs.last_hidden_state.mean(dim=1)
            embeddings.append(chunk_embeddings)
        return embeddings

embedder = Embedder()
chunk_embeddings = embedder.embed_text_chunks(all_chunks)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [17]:
class PineconeIntegration:
    def __init__(self):
        self.pc = Pinecone(api_key=PINECONE_API_KEY)

    def create_index(self, name='duke-chatbot', dimension=384, metric='cosine'):
        if name not in self.pc.list_indexes().names():
            self.pc.create_index(name=name, dimension=dimension, metric=metric,
                                 spec=ServerlessSpec(cloud='aws', region='us-west-2'))

    def upload_embeddings(self, index_name, chunk_embeddings):
        formatted_embeddings_gist = [{"id": str(i), "values": emb.tolist()[0], "metadata": {"text": str(text)}} for i, (emb, text) in enumerate(zip(chunk_embeddings, all_chunks))]
        index = self.pc.Index(index_name)
        batch_size = 100
        for i in range(0, len(formatted_embeddings_gist), batch_size):
            batch = formatted_embeddings_gist[i:i + batch_size]
            index.upsert(vectors=batch)
        print("Embeddings uploaded successfully!")


In [18]:
pinecone_integration = PineconeIntegration()
pinecone_integration.create_index()
pinecone_integration.upload_embeddings('duke-chatbot', chunk_embeddings)

Embeddings uploaded successfully!
